1 - as laboratorinis darbas (individualus)

Nojus Džiaugys s2110552

Pasirinktos klases: Jelyfish, Isopod, Snail

Modelis: resnet50

version 1.1

## Setup

In [70]:
import torch, torchvision
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
import numpy as np
import pprint


### Commons

#### Adding a device

In [3]:
def use_device():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    return device

#### Global variables

In [100]:
CHOSEN_CLASSES = sorted(["Jellyfish", "Snail", "Isopod"])
THRESHOLD = 0.7
BATCH_SIZE = 10
DATA_DIR = "data3"

### Download images

should only be run once, or when new classes get added

In [40]:
import os
from openimages.download import download_dataset

data_dir_base = "data"
number_for_samples = 1000
i = 1
while os.path.exists(f"{data_dir_base}{i}"):
    i += 1    
os.makedirs(f"{data_dir_base}{i}")

print("Download started...")
download_dataset(f"{data_dir_base}{i}", CHOSEN_CLASSES, limit=number_for_samples)

Download started...


2024-03-05  14:34:04 INFO Downloading 379 train images for class 'jellyfish'
100%|██████████| 379/379 [00:15<00:00, 24.62it/s]
2024-03-05  14:34:20 INFO Downloading 414 train images for class 'snail'
100%|██████████| 414/414 [00:14<00:00, 29.18it/s]
2024-03-05  14:34:34 INFO Downloading 72 train images for class 'isopod'
100%|██████████| 72/72 [00:03<00:00, 21.26it/s]
2024-03-05  14:34:39 INFO Downloading 23 validation images for class 'jellyfish'
100%|██████████| 23/23 [00:01<00:00, 11.54it/s]
2024-03-05  14:34:41 INFO Downloading 31 validation images for class 'snail'
100%|██████████| 31/31 [00:02<00:00, 15.05it/s]
2024-03-05  14:34:43 INFO Downloading 5 validation images for class 'isopod'
100%|██████████| 5/5 [00:01<00:00,  3.96it/s]
2024-03-05  14:34:48 INFO Downloading 54 test images for class 'jellyfish'
100%|██████████| 54/54 [00:03<00:00, 16.59it/s]
2024-03-05  14:34:51 INFO Downloading 100 test images for class 'snail'
100%|██████████| 100/100 [00:04<00:00, 20.03it/s]
2024-03

{'jellyfish': {'images_dir': 'data3\\jellyfish\\images'},
 'snail': {'images_dir': 'data3\\snail\\images'},
 'isopod': {'images_dir': 'data3\\isopod\\images'}}

### Model setup

#### Model itself

In [92]:
device = use_device()
model = torchvision.models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device)

Using device: cuda:0


#### Image transformer for model

In [93]:
all_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

#### Run model
for one image

In [101]:
image = Image.open('data1\spider\images\\0a03dbf1a69f2bec.jpg')

# Model runs on device
output = model(all_transforms(image).unsqueeze(0).to(device))
output = output.squeeze().cpu()
# idx = torch.argmax(output)
# print(torch.argmax(output))


classes = open('general_classification.txt', 'r').readlines()
# chatgpt
probabilities = torch.nn.functional.softmax(output, dim=0)
for class_name in CHOSEN_CLASSES:
    class_idx = classes.index((class_name + '\n').lower())
    class_prob = probabilities[class_idx]
    print(f"Class: {class_name}, Probability: {class_prob.item() * 100:.2f}%")


Class: Isopod, Probability: 0.95%
Class: Jellyfish, Probability: 0.06%
Class: Snail, Probability: 0.03%


### Metrics

#### Data loader
makes a dataset of all images

In [108]:
# Create an ImageFolder dataset
dataset = torchvision.datasets.ImageFolder(root=DATA_DIR, transform=all_transforms)

# Create a DataLoader for the dataset
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)


#### Calculates metrics
based on predictions and actual truths

In [106]:
def calculate_metrics(ground_truth, predictions, threshold=THRESHOLD):
    # Convert predictions to binary values based on the threshold
    predictions = (np.array(predictions) >= threshold).astype(np.float64)

    # Dictionary to store metrics for each chosen class
    metrics_per_class = {}

    # Lists to store True Positives, True Negatives, False Positives, and False Negatives for each class
    Tp = [0] * len(CHOSEN_CLASSES)
    Tn = [0] * len(CHOSEN_CLASSES)
    Fp = [0] * len(CHOSEN_CLASSES)
    Fn = [0] * len(CHOSEN_CLASSES)

    # Loop through each batch of ground truth and predictions
    for batch_truths, batch_predictions in zip(ground_truth, predictions):
        # Loop through each sample in the batch
        for i in range(len(batch_truths)):
            # Loop through each class
            for j in range(len(batch_truths[i])):
                # Update counts for True Positives, True Negatives, False Positives, and False Negatives
                Tp[j] += np.bitwise_and(batch_truths[i][j] == 1, batch_predictions[i][j] == 1)
                Tn[j] += np.bitwise_and(batch_truths[i][j] == 0, batch_predictions[i][j] == 0)
                Fp[j] += np.bitwise_and(batch_truths[i][j] == 0, batch_predictions[i][j] == 1)
                Fn[j] += np.bitwise_and(batch_truths[i][j] == 1, batch_predictions[i][j] == 0)

    # Calculate and store metrics for each chosen class
    for i in range(len(CHOSEN_CLASSES)):
        tp = Tp[i]
        tn = Tn[i]
        fp = Fp[i]
        fn = Fn[i]

        metrics_per_class[CHOSEN_CLASSES[i]] = {
            'accuracy': (tp + tn) / (tp + tn + fp + fn),
            'recall': (tp) / (tp + fn),
            'precision': (tp) / (tp + fp)
        }

        # Calculate F1 score and add it to the metrics dictionary
        m = metrics_per_class[CHOSEN_CLASSES[i]]
        m['f1'] = 2 * (m['precision'] * m['recall']) / (m['precision'] + m['recall'])

    return metrics_per_class


#### Run model on all images

In [107]:
# Initialize lists to store predictions and ground truth labels
all_predictions = []
all_ground_truth = []

# Set the model to evaluation mode
model.eval()

# Iterate through the DataLoader
for inputs, labels in dataset_loader:
    # Move inputs to the device
    inputs = inputs.to(device)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(inputs)

    # Apply softmax to the model's output along the specified dimension
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    
    # Convert model outputs to numpy array
    predictions = probabilities.squeeze().cpu().numpy()

    # Convert labels tensor to numpy array
    ground_truth_indices = labels.numpy()

    # Convert ground truth indices to one-hot encoded vectors
    ground_truth_one_hot = torch.nn.functional.one_hot(torch.tensor(ground_truth_indices), num_classes=len(CHOSEN_CLASSES)).numpy()

    # Filter predictions for chosen classes
    chosen_predictions = predictions[:, [classes.index(f"{class_name.lower()}" + "\n") for class_name in CHOSEN_CLASSES]]

    # Append predictions and ground truth to the lists
    all_predictions.append(chosen_predictions)
    all_ground_truth.append(ground_truth_one_hot)

# Pad all arrays to have the same number of rows (BATCH_SIZE)
all_predictions_padded = [np.pad(x, ((0, BATCH_SIZE - len(x)), (0, 0))) for x in all_predictions]

# Use calculate_metrics function
metrics = calculate_metrics(all_ground_truth, all_predictions_padded, threshold=THRESHOLD)

# Print results
print(f"Threshold: {THRESHOLD}\n")
for i in metrics:
    print(f"Class: {i}")
    pprint.pprint(metrics[i])
    print('\n')


Threshold: 0.7

Class: Isopod
{'accuracy': 0.925207756232687,
 'f1': 0.024096385542168676,
 'precision': 1.0,
 'recall': 0.012195121951219513}


Class: Jellyfish
{'accuracy': 0.6195752539242844,
 'f1': 0.176,
 'precision': 1.0,
 'recall': 0.09649122807017543}


Class: Snail
{'accuracy': 0.4986149584487535,
 'f1': 0.007312614259597807,
 'precision': 1.0,
 'recall': 0.003669724770642202}


